In [1]:
import sys
sys.path.append('../src')

import numpy as np 
import pickle

from FSMextractor import *

from sklearn.metrics import silhouette_score

BATCH_SIZE = 10
MODEL_FILE = "../experiments/boolean/models/boolean.h5"
TESTDATA_FILE = "../experiments/boolean/test_data_boolean.pkl" 

model = load_model(MODEL_FILE)
test_data = pickle.load(open(TESTDATA_FILE))


alphabet = set(["X", "&", "|", "1", "0"])

def arr2char(a):
    arr = a.tolist()
    if arr == [1,0,0,0,0]:
        return "x"
    if arr == [0,1,0,0,0]:
        return "&"
    if arr == [0,0,1,0,0]:
        return "0"
    if arr == [0,0,0,1,0]:
        return "1"
    if arr == [0,0,0,0,1]:
        return "|"
    else:
        raise ValueError("sorry, can't recognize label")
    


Using TensorFlow backend.


In [2]:
# kmeans discretizer
kMeans = kMeansQuantizer(5)

# getting the sample based transition extractor
sbtrans = SampleBasedTransExtractor(conv_func=arr2char)

experiment = FSMExtractor(model, test_data, quantization=kMeans, transition=sbtrans, minimization=None)

# get hidden states 
states = experiment.get_hidden_states()


# # add N random dimensions at the end

# num_symbols = states.shape[0]
# artificial_neurons = np.random.uniform(low=-1.0, high =1.0, size=(num_symbols,1))
# states_alt = np.hstack((states, artificial_neurons))


# # iterate between clustering and dimension elimination
# # eliminating the dimensions with the highest variance from the centroids


# fsm = []
# # XXX: number of neurons is default number of neurons (16) plus 1.
# neuron_nums = np.arange(17)
# silhouettes = []

# for i in range(16): # XXX: delete neurons until only 1 is left.
    
#     # printing separator
#     print("ITERATION {}".format(i))


#     # labels = experiment.quant_func.quantize(states_alt)
    
#     # generating and saving fsm's
    
# #     fsm["iteration #{}".format(i)] = experiment.cluster_transition(states_alt)
    
#     fsm.append(experiment.cluster_transition(states_alt))
    
#     # extracting macrostate predictions
#     labels = experiment.quant_func.getLabels()   
#     centroids = experiment.quant_func.getCentroids() 


#     # find out how well the hidden states are being clustered into macrostates
#     silh = experiment.quant_func.getSilhouetteCoef(states_alt)
#     silhouettes.append(silh)    
#     print("Sillhouette coefficient: {}".format(silh))

    
#     # finding a dimension to eliminate: the dimension that has the greatest average distance from
#     # centroids of clustering
#     # argmax_i \sum{j to k} 1/number_in_cluster * \sum{all i} x^(j)_i - C(j)(i) 
    
#     delta = states_alt - centroids[labels]
#     mean_delta = np.mean(delta, axis=0)

#     jettison = np.argmax(mean_delta)
#     print("Throwing out neuron number {}".format(neuron_nums[jettison]))
    
#     # throw out activations for that neuron in states_alt
#     neuron_nums = np.delete(neuron_nums, jettison)     
        
#     states_alt = np.delete(states_alt, jettison, axis=1)

BEWARE _ ONLY SUPPORTS CONSECUTIVE LAYER IDS STARTING AT 0
Creates spy models
... for id 0 : <keras.layers.recurrent.LSTM object at 0x109a03850>
Gets the activations for the hidden states


In [3]:
%matplotlib inline
import matplotlib.pyplot as plt

In [4]:
# # finding the standard deviation of the probabilities

# std_prob = []

# for (i, this_fsm) in enumerate(fsm):
#     this_std = np.std(this_fsm.values())
#     print("{}, {}".format(i, this_std))
#     std_prob.append(this_std)

    
# plt.title("Standard deviation of probabilities over trials")
# plt.plot(range(len(std_prob)),std_prob)
# plt.show()
    

In [5]:
# plt.title("Silhouttes coefs vs. std of probabilties")
# plt.scatter(silhouettes,std_prob)
# plt.show()

# from scipy.stats.stats import pearsonr

# print(pearsonr(silhouettes, std_prob))

# NOTES

next steps:

- some program to evaluate the accuracy of an FSM
- do this with an LSTM with 4 neurons instead of 8
- add 500+ artificial neurons
- bottom up approach, CLIQUE, ENCLUS

- baseline - clustering the columns of the activation matrix
- using mutual information, and then for each of those clusters
- find clusters within those subspaces

- use dni on fsm to interpret

- exhaustive approach, gain some more intuition, with pruning algorithms


- number of parentheses and plot it


- Make an assumption of what a "good" fsm looks like - clear clustering and high-entropy probability distribution
- for transitions

- Robustness - how far can a neuron deviate from the cluster centroids without breaking the FSM - RNN correspondence?


## Brute force approach to subspace extraction
Using a bottom-up exhaustive approach using the entropy-based subspace pruning method in http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.33.1465&rep=rep1&type=pdf. 

In [ ]:
import subclustering as sc
reload(sc)

# TODO: rm topbound once multiprocessing works
num_seqs = 80
topbound = 50*800
states = states[:topbound, :]
print(states.shape)

if __name__=="__main__":
    subspace_clusters = sc.bruteforce_prunesubspace(states, 
                                                    parallel=True,
                                                    num_process=4)

# subspace_clusters = sc.bruteforce_prunesubspace(states, 
#                                                 parallel=False)

(40000, 16)
multirpocess function (0,)
multirpocess function (1, 2, 7, 10, 11)
multirpocess function (2, 3, 7, 9, 12, 13)
multirpocess function (0, 2, 4, 7, 8, 15)
multirpocess function (1,)
multirpocess function (1, 2, 7, 10, 12)
multirpocess function (2, 3, 7, 9, 12, 14)
multirpocess function (2,)
multirpocess function (1, 2, 7, 10, 13)
multirpocess function (3,)
multirpocess function (0, 2, 4, 7, 9, 10)
multirpocess function (2, 3, 7, 9, 12, 15)
multirpocess function (1, 2, 7, 10, 14)
multirpocess function (4,)
multirpocess function (2, 3, 7, 9, 13, 14)
multirpocess function (5,)
multirpocess function (1, 2, 7, 10, 15)
multirpocess function (6,)
multirpocess function (0, 2, 4, 7, 9, 11)
multirpocess function (7,)
multirpocess function (2, 3, 7, 9, 13, 15)
multirpocess function (1, 2, 7, 11, 12)
multirpocess function (8,)
multirpocess function (2, 3, 7, 9, 14, 15)
multirpocess function (9,)
multirpocess function (1, 2, 7, 11, 13)
multirpocess function (10,)
multirpocess function (0, 

multirpocess function (0, 2, 4, 8, 10, 13)
multirpocess function (2, 3, 8, 11, 13, 15)
multirpocess function (3, 13)
multirpocess function (1, 2, 11, 13, 14)
multirpocess function (3, 14)
multirpocess function (1, 2, 11, 13, 15)
multirpocess function (2, 3, 8, 11, 14, 15)
multirpocess function (3, 15)
multirpocess function (0, 2, 4, 8, 10, 14)
multirpocess function (1, 2, 11, 14, 15)
multirpocess function (2, 3, 8, 12, 13, 14)
multirpocess function (4, 5)
multirpocess function (1, 2, 12, 13, 14)
multirpocess function (4, 6)
multirpocess function (2, 3, 8, 12, 13, 15)
multirpocess function (0, 2, 4, 8, 10, 15)
multirpocess function (1, 2, 12, 13, 15)
multirpocess function (2, 3, 8, 12, 14, 15)
multirpocess function (4, 7)
multirpocess function (1, 2, 12, 14, 15)
multirpocess function (4, 8)
multirpocess function (2, 3, 8, 13, 14, 15)
multirpocess function (0, 2, 4, 8, 11, 12)
multirpocess function (1, 2, 13, 14, 15)
multirpocess function (4, 9)
multirpocess function (2, 3, 9, 10, 11, 12

multirpocess function (2, 4, 5, 6, 8, 11)
multirpocess function (1, 3, 5, 6, 7)
multirpocess function (0, 1, 5)
multirpocess function (1, 3, 5, 6, 8)
multirpocess function (2, 4, 5, 6, 8, 12)
multirpocess function (0, 1, 6)
multirpocess function (1, 3, 5, 6, 9)
multirpocess function (0, 2, 4, 10, 12, 13)
multirpocess function (2, 4, 5, 6, 8, 13)
multirpocess function (1, 3, 5, 6, 10)
multirpocess function (0, 1, 7)
multirpocess function (1, 3, 5, 6, 11)
multirpocess function (2, 4, 5, 6, 8, 14)
multirpocess function (0, 2, 4, 10, 12, 14)
multirpocess function (0, 1, 8)
multirpocess function (1, 3, 5, 6, 12)
multirpocess function (1, 3, 5, 6, 13)
multirpocess function (2, 4, 5, 6, 8, 15)
multirpocess function (0, 1, 9)
multirpocess function (1, 3, 5, 6, 14)
multirpocess function (0, 2, 4, 10, 12, 15)
multirpocess function (2, 4, 5, 6, 9, 10)
multirpocess function (0, 1, 10)
multirpocess function (1, 3, 5, 6, 15)
multirpocess function (2, 4, 5, 6, 9, 11)
multirpocess function (1, 3, 5, 7

multirpocess function (1, 3, 6, 11, 12)
multirpocess function (0, 5, 9)
multirpocess function (2, 4, 5, 7, 13, 14)
multirpocess function (1, 3, 6, 11, 13)
multirpocess function (0, 2, 5, 6, 10, 12)
multirpocess function (0, 5, 10)
multirpocess function (1, 3, 6, 11, 14)
multirpocess function (2, 4, 5, 7, 13, 15)
multirpocess function (0, 2, 5, 6, 10, 13)
multirpocess function (0, 5, 11)
multirpocess function (1, 3, 6, 11, 15)
multirpocess function (2, 4, 5, 7, 14, 15)
multirpocess function (0, 5, 12)
multirpocess function (0, 2, 5, 6, 10, 14)
multirpocess function (1, 3, 6, 12, 13)
multirpocess function (0, 5, 13)
multirpocess function (2, 4, 5, 8, 9, 10)
multirpocess function (1, 3, 6, 12, 14)
multirpocess function (0, 2, 5, 6, 10, 15)
multirpocess function (0, 5, 14)
multirpocess function (1, 3, 6, 12, 15)
multirpocess function (2, 4, 5, 8, 9, 11)
multirpocess function (0, 2, 5, 6, 11, 12)
multirpocess function (0, 5, 15)
multirpocess function (1, 3, 6, 13, 14)
multirpocess function 

multirpocess function (1, 3, 9, 12, 13)
multirpocess function (0, 2, 5, 8, 9, 12)
multirpocess function (2, 4, 5, 10, 13, 15)
multirpocess function (1, 3, 9, 12, 14)
multirpocess function (0, 13, 14)
multirpocess function (0, 2, 5, 8, 9, 13)
multirpocess function (2, 4, 5, 10, 14, 15)
multirpocess function (1, 3, 9, 12, 15)
multirpocess function (0, 13, 15)
multirpocess function (2, 4, 5, 11, 12, 13)
multirpocess function (1, 3, 9, 13, 14)
multirpocess function (0, 2, 5, 8, 9, 14)
multirpocess function (1, 3, 9, 13, 15)
multirpocess function (0, 14, 15)
multirpocess function (2, 4, 5, 11, 12, 14)
multirpocess function (0, 2, 5, 8, 9, 15)
multirpocess function (1, 3, 9, 14, 15)
multirpocess function (2, 4, 5, 11, 12, 15)
multirpocess function (0, 2, 5, 8, 10, 11)
multirpocess function (1, 3, 10, 11, 12)
multirpocess function (1, 2, 3)
multirpocess function (1, 2, 4)
multirpocess function (1, 3, 10, 11, 13)
multirpocess function (2, 4, 5, 11, 13, 14)
multirpocess function (0, 2, 5, 8, 10

...

multirpocess function (2, 5, 6, 8, 9, 11, 15)
multirpocess function (0, 4, 5, 6, 7, 9, 15)
multirpocess function (0, 2, 7, 10, 11, 12, 13, 14)
multirpocess function (2, 5, 6, 8, 9, 12, 13)
multirpocess function (0, 1, 3, 4, 7, 10, 11, 14)
multirpocess function (0, 4, 5, 6, 7, 10, 11)
multirpocess function (2, 5, 6, 8, 9, 12, 14)
multirpocess function (0, 2, 7, 10, 11, 12, 13, 15)
multirpocess function (0, 1, 3, 4, 7, 10, 11, 15)
multirpocess function (2, 5, 6, 8, 9, 12, 15)
multirpocess function (0, 4, 5, 6, 7, 10, 12)
multirpocess function (0, 2, 7, 10, 11, 12, 14, 15)
multirpocess function (2, 5, 6, 8, 9, 13, 14)
multirpocess function (0, 1, 3, 4, 7, 10, 12, 13)
multirpocess function (0, 4, 5, 6, 7, 10, 13)
multirpocess function (2, 5, 6, 8, 9, 13, 15)
multirpocess function (0, 2, 7, 10, 11, 13, 14, 15)
multirpocess function (0, 1, 3, 4, 7, 10, 12, 14)
multirpocess function (2, 5, 6, 8, 9, 14, 15)
multirpocess function (0, 4, 5, 6, 7, 10, 14)
multirpocess function (2, 5, 6, 8, 10, 11

multirpocess function (0, 1, 3, 4, 8, 10, 11, 14)
multirpocess function (2, 5, 6, 12, 13, 14, 15)
multirpocess function (0, 3, 4, 5, 6, 7, 8, 12)
multirpocess function (2, 5, 7, 8, 9, 10, 11)
multirpocess function (0, 4, 5, 6, 9, 10, 11)
multirpocess function (0, 1, 3, 4, 8, 10, 11, 15)
multirpocess function (0, 3, 4, 5, 6, 7, 8, 13)
multirpocess function (2, 5, 7, 8, 9, 10, 12)
multirpocess function (0, 4, 5, 6, 9, 10, 12)
multirpocess function (2, 5, 7, 8, 9, 10, 13)
multirpocess function (0, 3, 4, 5, 6, 7, 8, 14)
multirpocess function (0, 1, 3, 4, 8, 10, 12, 13)
multirpocess function (2, 5, 7, 8, 9, 10, 14)
multirpocess function (0, 3, 4, 5, 6, 7, 8, 15)
multirpocess function (0, 4, 5, 6, 9, 10, 13)
multirpocess function (2, 5, 7, 8, 9, 10, 15)
multirpocess function (0, 1, 3, 4, 8, 10, 12, 14)
multirpocess function (0, 3, 4, 5, 6, 7, 9, 10)
multirpocess function (2, 5, 7, 8, 9, 11, 12)
multirpocess function (0, 4, 5, 6, 9, 10, 14)
multirpocess function (0, 1, 3, 4, 8, 10, 12, 15)
mu

multirpocess function (0, 1, 3, 4, 9, 12, 13, 15)
multirpocess function (2, 5, 7, 10, 11, 13, 14)
multirpocess function (0, 3, 4, 5, 6, 8, 13, 14)
multirpocess function (2, 5, 7, 10, 11, 13, 15)
multirpocess function (0, 4, 5, 7, 8, 9, 10)
multirpocess function (0, 1, 3, 4, 9, 12, 14, 15)
multirpocess function (0, 3, 4, 5, 6, 8, 13, 15)
multirpocess function (2, 5, 7, 10, 11, 14, 15)
multirpocess function (2, 5, 7, 10, 12, 13, 14)
multirpocess function (0, 4, 5, 7, 8, 9, 11)
multirpocess function (0, 3, 4, 5, 6, 8, 14, 15)
multirpocess function (0, 1, 3, 4, 9, 13, 14, 15)
multirpocess function (2, 5, 7, 10, 12, 13, 15)
multirpocess function (0, 4, 5, 7, 8, 9, 12)
multirpocess function (0, 3, 4, 5, 6, 9, 10, 11)
multirpocess function (2, 5, 7, 10, 12, 14, 15)
multirpocess function (0, 1, 3, 4, 10, 11, 12, 13)
multirpocess function (2, 5, 7, 10, 13, 14, 15)
multirpocess function (0, 3, 4, 5, 6, 9, 10, 12)
multirpocess function (0, 4, 5, 7, 8, 9, 13)
multirpocess function (0, 1, 3, 4, 10,

multirpocess function (0, 4, 5, 7, 9, 13, 15)
multirpocess function (0, 3, 4, 5, 7, 8, 9, 15)
multirpocess function (2, 5, 9, 11, 12, 14, 15)
multirpocess function (0, 1, 3, 5, 6, 7, 10, 15)
multirpocess function (2, 5, 9, 11, 13, 14, 15)
multirpocess function (0, 3, 4, 5, 7, 8, 10, 11)
multirpocess function (0, 4, 5, 7, 9, 14, 15)
multirpocess function (2, 5, 9, 12, 13, 14, 15)
multirpocess function (0, 1, 3, 5, 6, 7, 11, 12)
multirpocess function (0, 3, 4, 5, 7, 8, 10, 12)
multirpocess function (2, 5, 10, 11, 12, 13, 14)
multirpocess function (0, 4, 5, 7, 10, 11, 12)
multirpocess function (0, 1, 3, 5, 6, 7, 11, 13)
multirpocess function (0, 3, 4, 5, 7, 8, 10, 13)
multirpocess function (2, 5, 10, 11, 12, 13, 15)
multirpocess function (2, 5, 10, 11, 12, 14, 15)
multirpocess function (0, 4, 5, 7, 10, 11, 13)
multirpocess function (0, 3, 4, 5, 7, 8, 10, 14)
multirpocess function (0, 1, 3, 5, 6, 7, 11, 14)
multirpocess function (2, 5, 10, 11, 13, 14, 15)
multirpocess function (0, 3, 4, 5,

multirpocess function (0, 4, 5, 8, 10, 11, 13)
multirpocess function (0, 1, 3, 5, 6, 9, 11, 13)
multirpocess function (2, 6, 7, 9, 10, 11, 15)
multirpocess function (0, 3, 4, 5, 7, 12, 13, 15)
multirpocess function (0, 4, 5, 8, 10, 11, 14)
multirpocess function (0, 1, 3, 5, 6, 9, 11, 14)
multirpocess function (2, 6, 7, 9, 10, 12, 13)
multirpocess function (0, 3, 4, 5, 7, 12, 14, 15)
multirpocess function (0, 1, 3, 5, 6, 9, 11, 15)
multirpocess function (0, 4, 5, 8, 10, 11, 15)
multirpocess function (2, 6, 7, 9, 10, 12, 14)
multirpocess function (0, 3, 4, 5, 7, 13, 14, 15)
multirpocess function (0, 3, 4, 5, 8, 9, 10, 11)
multirpocess function (0, 1, 3, 5, 6, 9, 12, 13)
multirpocess function (0, 4, 5, 8, 10, 12, 13)
multirpocess function (2, 6, 7, 9, 10, 12, 15)
multirpocess function (0, 3, 4, 5, 8, 9, 10, 12)
multirpocess function (0, 1, 3, 5, 6, 9, 12, 14)
multirpocess function (0, 4, 5, 8, 10, 12, 14)
multirpocess function (2, 6, 7, 9, 10, 13, 14)
multirpocess function (0, 3, 4, 5, 8,

In [8]:
if __name__=="__main__":
    print("hello world")

hello world


In [11]:
# get a list of entropy values
data = [ent[1] for ent in subspace_clusters]

top_fifth_ent = np.percentile(data, 80)
top_quart_ent = np.percentile(data, 75)
top_95_ent = np.percentile(data, 95)

print("maximum entropy = {}".format(max(data)))
print("75 percentile cutoff: {}".format(top_quarter_ent))

data = [subspace_clusters[val][1] for val in subspace_clusters]
values, base = np.histogram(data, bins=40)
cumulative = np.cumsum(values)
plt.title("Cumulative distribution for accepted subspaces.")
plt.plot(base[:-1], cumulative, c="blue")
plt.plot(base[:-1], len(data)-cumulative, c="green")
plt.plot([top_95_ent, top_95_ent], [350,0], c="red")
plt.xlabel("entropy")


NameError: name 'data' is not defined

In [2]:
reload(sc)

## Experiment to see if filtering when noise is added.
N_random = 50

rand_activations = np.random.random((states.shape[0], N_random))
alt_states = np.hstack((states, rand_activations))
print("shape of artificial activation matrix: {}".format(alt_states.shape))

subspace_clusters = sc.bruteforce_prunesubspace(alt_states, 
                                                sc.entropy_based, 
                                                sc.kmeans_wrapper, 
                                                top_fifth_ent,
                                               delt=0.2) # increased delt 

NameError: name 'sc' is not defined

In [ ]:
## checking if any of the artificial neurons made it into the top

# turn dictionary into a list, and then rank subspaces according to the 



## Transition extraction
Evaluation using gini coefficient
